# Modelo

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [ ]:
base_path = '/home/cburich_pymnts/buckets/b1/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'


# base_path = 'C:/Users/Cristian Burich/Desktop/MA/segundo/eyf/'
# dataset_path = base_path + 'datasets/'
# modelos_path = base_path + 'modelos/'
# db_path = base_path + 'db/'


dataset_file = 'competencia_03_fe_U_k300.parquet' 

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [165229,165211,165203,165237,165247]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [ ]:
data = data.drop(columns=['clase_ternaria_1', 'tmobile_app', 'cmobile_app_trx', 'foto_mes'])

In [ ]:
# Asegúrate de que ctrx_quarter es numérico
data['ctrx_quarter_normalizado'] = pd.to_numeric(data['ctrx_quarter'], errors='coerce')

# Aplicar las condiciones para normalizar ctrx_quarter
data.loc[data['cliente_antiguedad'] == 1, 'ctrx_quarter_normalizado'] = data['ctrx_quarter'] * 5
data.loc[data['cliente_antiguedad'] == 2, 'ctrx_quarter_normalizado'] = data['ctrx_quarter'] * 2
data.loc[data['cliente_antiguedad'] == 3, 'ctrx_quarter_normalizado'] = data['ctrx_quarter'] * 1.2

In [ ]:
data.shape

In [ ]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202007, 202008,
               202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202106, 202107] # dejo afuera 202006 por que es horrible


(376300, 679)

In [5]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [ ]:
train_data = data[data['foto_mes'].isin(meses_train)]

X_train = train_data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria1','clase_binaria2', 'foto_mes'], axis=1)
y_train_binaria1 = train_data['clase_binaria1'] # Solo BAJA+2
y_train_binaria2 = train_data['clase_binaria2'] # junta a los 2 BAJA
w_train = train_data['clase_peso']

In [7]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [ ]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150), 
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), 
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria2, # eligir la clase
                              weight=w_train)   

    
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=100, 
          #  early_stopping_rounds= int(50 + 5 / learning_rate),
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [ ]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia3_lgbm_k302" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-17 20:38:17,209] Using an existing study with name 'competencia2_lgbm_v08' instead of creating a new one.


In [ ]:
study.optimize(objective, n_trials=100) # Habia hecho 30, hago 70 mas porque parece que es un modelo superior

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[550]	cv_agg's valid gan_eval: 6.62284e+08 + 1.61155e+06


[I 2024-11-17 20:42:22,083] Trial 30 finished with value: 3311420000.0 and parameters: {'num_leaves': 102, 'learning_rate': 0.02212150442747906, 'min_data_in_leaf': 708, 'feature_fraction': 0.2987543446016156, 'bagging_fraction': 0.6157890963391109}. Best is trial 22 with value: 3319092000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[527]	cv_agg's valid gan_eval: 6.59637e+08 + 4.10708e+06


[I 2024-11-17 20:45:02,999] Trial 31 finished with value: 3298183000.0 and parameters: {'num_leaves': 56, 'learning_rate': 0.014310366137917837, 'min_data_in_leaf': 1305, 'feature_fraction': 0.21008685324297327, 'bagging_fraction': 0.9150656789105077}. Best is trial 22 with value: 3319092000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[837]	cv_agg's valid gan_eval: 6.63186e+08 + 3.6387e+06


...

[I 2024-11-18 00:44:56,469] Trial 97 finished with value: 3313065000.0 and parameters: {'num_leaves': 143, 'learning_rate': 0.013273659401895083, 'min_data_in_leaf': 939, 'feature_fraction': 0.1701928153524901, 'bagging_fraction': 0.3863542720955415}. Best is trial 22 with value: 3319092000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[525]	cv_agg's valid gan_eval: 6.61973e+08 + 3.31315e+06


[I 2024-11-18 00:50:01,896] Trial 98 finished with value: 3309866000.0 and parameters: {'num_leaves': 134, 'learning_rate': 0.017891131466557995, 'min_data_in_leaf': 1327, 'feature_fraction': 0.6668838533905063, 'bagging_fraction': 0.25934076625067126}. Best is trial 22 with value: 3319092000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[888]	cv_agg's valid gan_eval: 6.60765e+08 + 2.17508e+06


[I 2024-11-18 00:53:09,380] Trial 99 finished with value: 3303825000.0 and parameters: {'num_leaves': 141, 'learning_rate': 0.015269812774196912, 'min_data_in_leaf': 1020, 'feature_fraction': 0.10024518118924586, 'bagging_fraction': 0.9734649133265478}. Best is trial 22 with value: 3319092000.0.


Analizamos los resultados as usual

In [11]:
optuna.visualization.plot_optimization_history(study)

In [12]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [13]:
plot_slice(study)

In [14]:
plot_contour(study)

In [15]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [16]:
study.best_trial.params

{'num_leaves': 95,
 'learning_rate': 0.014790793124814124,
 'min_data_in_leaf': 1162,
 'feature_fraction': 0.32039319093779284,
 'bagging_fraction': 0.7236519946486292}

In [17]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

871